## Plots for loop tests for which all subcats have the same number of a single LID

In [1]:
# IMPORTANT:  RUN THIS BLOCK FIRST
# ONLY for loop tests for which all subcats have the same number of a single LID:
%matplotlib inline
from pymongo import MongoClient
from datetime import datetime
import matplotlib.pyplot as plt
def volume_reduction_and_runoff_volume_vs_lid_number(collectionName):
    mGal = 133680.5  # 1 Million gallon in cubic feet
    runs = db[collectionName]
    cursor = runs.find()
    numList = []
    volReductionList = []
    totalSurfaceOutflowList = []
    csoVolList=[]
    noLidRun = cursor[0]
    noLidVolume = noLidRun['volume']
    for run in cursor:
        runParamsZero = run['runParamList'][0] 
        number = runParamsZero['Number']
        numList.append(number)
        volume = run['volume']
        volReduction = noLidVolume-volume
        volReductionList.append(volReduction)
        cso=run['cso_volume']
        csoVolList.append(cso)
        if run['lidDict'] == None:
            totalSurfaceOutflowList.append(None)
            continue
        else:
            totalSurfaceOutflow = 0.0
            runParamList = run['runParamList']
            lidDict = run['lidDict']
            for lidUsage in runParamList:
                lid = lidUsage['LID']
                subcat = lidUsage['Subcat']
                lidNumber = lidUsage['Number']
                #print lidNumber,
                lidArea = lidUsage['Area']  # in square feet
                lidKey = subcat+' '+lid
                surfaceOutflowInches = lidDict[lidKey]['Surface Outflow']
                surfaceOutflow = surfaceOutflowInches*lidNumber*lidArea/12.0  # cubic feet
                totalSurfaceOutflow += surfaceOutflow/mGal  # convert to million Gal/year
                
            #print totalSurfaceOutflow
            totalSurfaceOutflowList.append(totalSurfaceOutflow)


    return(numList, csoVolList, volReductionList, totalSurfaceOutflowList)

### Show the collections that have been generated

In [2]:
from pymongo import MongoClient

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "ewri2017"
username = "arthur"
password = "jXlIkLfu5Dfr"
authenticationDatabase = "admin"
client = MongoClient(mongoServer)
################################################

db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
collectionNames

[u'2008annual_1', u'july', u'july2008_1']

### Plot Results from Increasing numbers of Wakefield Raingardens in S1 - S6

In [3]:
collectionName = '2008annual_1'
print db[collectionName]
x = volume_reduction_and_runoff_volume_vs_lid_number(collectionName)
print x
(fig, ax1) = plt.subplots()
ax1.plot(x[0],x[1],'ko',markersize = 5)
ax1.set_xlabel("Number of LID",fontsize=12)
ax1.set_ylabel("CSO Volume Reduction (Mgal/yr)",fontsize=12)
ax2 = ax1.twinx()
ax2.plot(x[0],x[2],'ro',markersize = 5)
ax2.set_ylabel("Runoff Volume Reduction (Mgal/yr)",fontsize=12)
plt.title('Total Lid in Wingohocking',fontsize=20)


Collection(Database(MongoClient(host=['aws-2.greenphilly.net:27017'], document_class=dict, tz_aware=False, connect=True), u'ewri2017'), u'2008annual_1')


KeyError: 'cso_volume'